# Anomaly Detection: STEP1

## Outline
1. Introduction

2. Import Module

3. Load Dataset
    
    3.1 Description
    
    3.2 Define Anomalies
    
4. Split Dataset(Train, Test)

5. Exploratory Data Analysis

6. Implementation Anomaly Detection

## Introduction

[UCI machine learning repository](https://archive.ics.uci.edu/ml/index.php)의 갑상선 질환 데이터 셋을 이용하여 Point Anomalies를 탐지 구현을 목표로 하는 커널이다. 

데이터는 7200개의 행과 15개의 범주형 변수, 6개의 연속형 변수로 구성되어 있다. 이상 탐지 구현은 6개의 연속형 변수를 이용하여 구현할 것이다. 

라벨(class)은 정상데이터(Normal), 기능 항진(Hyperfunction)과 비정상(Subnormal)이 존재한다. 

훈련 데이터와 테스트 데이터로 분할되어 있지 않기 때문에 가장 먼저 분할을 수행한다.

## Import Module

In [2]:
import numpy as np
import pandas as pd
import scipy.io

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

## Load Dataset

In [3]:
dataset = pd.read_csv('../dataset/thyroid_disease/thyroid_disease.csv')
dataset = dataset[['Age', 'TSH', 'T3', 'TT4', 'T4U', 'FTI', 'class']]
print('Dataset Shape:',dataset.shape)
dataset.head()

Dataset Shape: (7200, 7)


,Age,TSH,T3,TT4,T4U,FTI,class
0,0.73,0.00060,0.015,0.120,0.082,0.146,3
1,0.24,0.00025,0.030,0.143,0.133,0.108,3
2,0.47,0.00190,0.024,0.102,0.131,0.078,3
3,0.64,0.00090,0.017,0.077,0.090,0.085,3
4,0.23,0.00025,0.026,0.139,0.090,0.153,3


In [4]:
dataset.describe()

,Age,TSH,T3,TT4,T4U,FTI,class
count,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000,7200.000000
mean,0.520518,0.004861,0.019977,0.109430,0.097838,0.113215,2.902778
std,0.189269,0.022979,0.007539,0.035969,0.019085,0.036098,0.365923
min,0.010000,0.000000,0.000500,0.002000,0.017000,0.002000,1.000000
25%,0.370000,0.000710,0.017000,0.089000,0.087000,0.095000,3.000000
50%,0.550000,0.001700,0.020100,0.107000,0.097000,0.111000,3.000000
75%,0.670000,0.002700,0.022000,0.125000,0.104000,0.127000,3.000000
max,0.970000,0.530000,0.180000,0.600000,0.233000,0.642000,3.000000


### Description
연속형 변수와 라벨로 구성된 데이터의 설명은 다음과 같다.
- Age: 연령
- TSH: 갑상선 자극 호르몬 검사 수치
    - 뇌하수체에서 생성되는 호르몬 중 하나. 뇌하수체가 제대로 작동하지 않으면 TSH가 상승하거나 감소하게 된다. 
    - TSH 농도가 증가하면, 갑상선이 비정상적인 양의 T4(티록신),T3(트리요오드티로닌)의 생성과 분비를 자극한다.  
    - 시상하부, 뇌하수체, 갑상선 3개의 기관이 모두 정상적인 기능을 하고 있을 때, 혈중 갑상선 호르몬을 일정하기 유지하기 위해 호르몬 생산을 제어한다.
- T3: 트리요오드티로닌 검사 수치
- TT4: 티록신 검사 수치
- T4U: 티록신 흡수율 검사 수치
- FTI: 유리 티록신 지수
    - 총 티록신 수치에 영향을 줄 수 있는 특정 단백질의 변화를 교정하는 갑상선 기능을 평가하는 방법 중 하나.

다음 표는 검사 결과와 검사 수치에 대한 의미를 요약해 놓은 것이다.

<img src="../image/TSH_T3_T4.png" width="500">

Source: [Lab Tests Online](https://labtestsonline.kr/)

### Define Anomalies
데이터의 라벨은 총 3가지로 나뉘어 있다. 이상 감지는 일반적인 데이터 패턴에서 벗어나는 값을 찾는 것이기 때문에 대부분 Class Imblance 문제를 가지고 있다. 따라서 이상 클래스를 다음으로 정의한다.
- 1, 2: Hyperthyroidism(Anomalies)
- 3: Normal

In [5]:
dataset['outlier'] = dataset['class'].apply(lambda x: 0 if x == 3 else 1)
dataset.drop(columns = 'class', inplace = True)

X = dataset.drop(columns = 'outlier')
y = dataset['outlier']

## Split Dataset
이제, 데이터를 훈련 데이터와 테스트 데이터로 분할한다. 

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

## Exploratory Data Analysis
훈련 데이터에 대한 탐색적 자료 분석을 수행한다. 

분석을 수행하기 전 예상할 수 있는 가정은 다음과 같다.
- 정상 데이터와 이상 데이터의 컬럼별 분포는 상이할 것이다.